## Codigo con MINIMA VARIANZA, POR LOS 3 METODOS SLSQP, MONTECARLO Y COBYLA

In [10]:
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
# pip install optuna
import optuna
import warnings
import logging

# Configurar el nivel de registro de Optuna para que sea más alto que INFO
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Suprimir las advertencias específicas
warnings.filterwarnings('ignore', message='A new study created in memory with name:')
warnings.filterwarnings('ignore', message='Method COBYLA cannot handle bounds.')

In [19]:
class EstrategiaOptimizada:
    def __init__(self, tickers=None, rf=None, lower_bound=0.10, higher_bound=0.99, start_date=None, end_date=None):
        # lower_bound=0.10, higher_bound=0.90,
        self.tickers = tickers
        self.rf = rf
        self.lower_bound = lower_bound
        self.higher_bound = higher_bound
        self.start_date = start_date
        self.end_date = end_date
        self.datos = self.cargar_datos()
        self.rendimientos = self.calcular_rendimientos()
        self.pesos_optimos = None
        self.optimization_strategy = None
        self.optimization_model = None

    def set_optimization_strategy(self, strategy):
        self.optimization_strategy = strategy

    def set_optimization_model(self, model):
        self.optimization_model = model

    def cargar_datos(self):
        if not self.tickers:
            raise ValueError("Debe proporcionar una lista de tickers.")
        return yf.download(self.tickers, start=self.start_date, end=self.end_date)['Adj Close']

    def calcular_rendimientos(self):
        return self.datos.pct_change().dropna()
    

    def optimizar_slsqp(self):
        if self.optimization_strategy == 'Minima Varianza':
            objetivo = self.minima_varianza
        elif self.optimization_strategy == 'Omega Ratio':
            objetivo = lambda pesos: -self.omega_ratio(pesos, self.rf)
        elif self.optimization_strategy == 'Semivarianza':
            objetivo = self.semivarianza
        elif self.optimization_strategy == 'Martingala':
            objetivo = self.martingala
        elif self.optimization_strategy == 'Roy Safety First Ratio':
             objetivo = self.roy_safety_first_ratio
        elif self.optimization_strategy == 'Sortino Ratio':
             objetivo = self.sortino_ratio
        elif self.optimization_strategy == 'CVAR':
            objetivo = self.cvar
        else:
            raise ValueError("Estrategia de optimización no válida.")
        
        restricciones = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        limites = [(self.lower_bound, self.higher_bound) for _ in self.tickers]
        
        resultado = minimize(objetivo, np.ones(len(self.tickers)) / len(self.tickers), method='SLSQP', bounds=limites, constraints=restricciones)
        self.pesos_optimos = resultado.x if resultado.success else None



    def optimizar_montecarlo(self):
        if self.optimization_strategy == 'Minima Varianza':
            objetivo = self.minima_varianza
        elif self.optimization_strategy == 'Omega Ratio':
            objetivo = lambda pesos: -self.omega_ratio(pesos, self.rf)
        elif self.optimization_strategy == 'Semivarianza':
            objetivo = self.semivarianza
        elif self.optimization_strategy == 'Martingala':
            objetivo = self.martingala
        elif self.optimization_strategy == 'Roy Safety First Ratio':
             objetivo = self.roy_safety_first_ratio
        elif self.optimization_strategy == 'Sortino Ratio':
             objetivo = self.sortino_ratio
        elif self.optimization_strategy == 'CVAR':
            objetivo = self.cvar
        else:
            raise ValueError("Estrategia de optimización no válida.")

        def optuna_objetivo(trial):
            pesos = np.array([trial.suggest_float(f'w{i}', self.lower_bound, self.higher_bound) for i in range(len(self.tickers))])
            pesos /= np.sum(pesos)  # Asegurar que los pesos sumen a 1
            return objetivo(pesos)

        estudio = optuna.create_study(direction='minimize')
        estudio.optimize(optuna_objetivo, n_trials=1000)
        self.pesos_optimos = [estudio.best_params[f'w{i}'] for i in range(len(self.tickers))]
        self.pesos_optimos = np.array(self.pesos_optimos) / np.sum(self.pesos_optimos)



    def optimizar_cobyla(self):
        if self.optimization_strategy == 'Minima Varianza':
            objetivo = self.minima_varianza
        elif self.optimization_strategy == 'Omega Ratio':
            objetivo = lambda pesos: -self.omega_ratio(pesos, self.rf)
        elif self.optimization_strategy == 'Semivarianza':
            objetivo = self.semivarianza
        elif self.optimization_strategy == 'Martingala':
            objetivo = self.martingala
        elif self.optimization_strategy == 'Roy Safety First Ratio':
             objetivo = self.roy_safety_first_ratio
        elif self.optimization_strategy == 'Sortino Ratio':
            objetivo = self.sortino_ratio
        elif self.optimization_strategy == 'CVAR':
            objetivo = self.cvar
        else:
            raise ValueError("Estrategia de optimización no válida.")

        # Establece un peso mínimo para asegurar que todos los tickers tengan algún peso
        min_weight = 0  # Ajusta este valor según sea necesario

        # Restricciones de desigualdad para asegurar que cada peso sea mayor que el peso mínimo
        restricciones = [{'type': 'ineq', 'fun': lambda pesos, i=i: pesos[i] - min_weight} for i in range(len(self.tickers))]

        # Añadir restricciones para asegurar que la suma de los pesos sea 1, permitiendo un pequeño margen de flexibilidad
        restricciones.append({'type': 'ineq', 'fun': lambda pesos: 1 - np.sum(pesos)})
        restricciones.append({'type': 'ineq', 'fun': lambda pesos: np.sum(pesos) - (1 - len(self.tickers) * min_weight)})

        # Generar pesos iniciales aleatorios
        pesos_iniciales = np.random.uniform(min_weight, 1, len(self.tickers))
        pesos_iniciales /= np.sum(pesos_iniciales)  # Normalizar para que la suma sea 1

        resultado = minimize(objetivo, pesos_iniciales, method='COBYLA', constraints=restricciones, options={'maxiter': 5000, 'catol': 0.01})
        if resultado.success:
            self.pesos_optimos = resultado.x / np.sum(resultado.x)
        else:
            self.pesos_optimos = None

    def minima_varianza(self, pesos):
        """Estrategia de mínima varianza."""
        return np.dot(pesos.T, np.dot(self.rendimientos.cov() * 252, pesos))

    def omega_ratio(self, pesos, threshold=0.0):
        """Estrategia basada en el Omega Ratio."""
        rendimientos_portafolio = np.dot(self.rendimientos, pesos)
        rendimientos_exceso = rendimientos_portafolio - threshold

        ganancia = np.sum(rendimientos_exceso[rendimientos_exceso > 0])
        perdida = -np.sum(rendimientos_exceso[rendimientos_exceso < 0])

        if perdida == 0:
            return np.inf  # Evitar división por cero
        return ganancia / perdida
    
    def semivarianza(self, pesos):
        """Estrategia de semivarianza."""
        rendimientos_negativos = self.rendimientos.copy()
        rendimientos_negativos[rendimientos_negativos > 0] = 0
        semivar = np.dot(pesos.T, np.dot(rendimientos_negativos.cov() * 252, pesos))
        return np.sqrt(semivar)

    
    def martingala(self, pesos):
        """Estrategia de Martingala."""
        rendimientos_pasados = self.rendimientos.iloc[-1]  # Últimos rendimientos
        ajuste = 1 - rendimientos_pasados / rendimientos_pasados.mean()  # Ajustar basado en el rendimiento relativo
        pesos_ajustados = pesos * ajuste
        return np.dot(pesos_ajustados.T, np.dot(self.rendimientos.cov() * 252, pesos_ajustados))
    
    def roy_safety_first_ratio(self, pesos):
        """Estrategia de Roy's Safety-First Ratio."""
        rendimiento_esperado = np.dot(self.rendimientos.mean() * 252, pesos)
        volatilidad = np.sqrt(np.dot(pesos.T, np.dot(self.rendimientos.cov() * 252, pesos)))
        return -(rendimiento_esperado - self.rf) / volatilidad
    
    def sortino_ratio(self, pesos):
            rendimientos_portafolio = np.dot(self.rendimientos, pesos)
            rendimiento_exceso = rendimientos_portafolio - self.rf / 252
            downside_deviation = np.sqrt(np.mean(np.minimum(rendimiento_exceso, 0) ** 2))
            return -(np.mean(rendimiento_exceso) / downside_deviation if downside_deviation != 0 else np.inf)
    
    def cvar(self, pesos, alpha=0.05):
        """
        Calcula el CVaR (Conditional Value at Risk) del portafolio.

        Parámetros:
        - pesos (np.array): Pesos del portafolio.
        - alpha (float): Nivel de confianza para el cálculo del CVaR.

        Retorna:
        - float: CVaR del portafolio.
        """
        rendimientos_portafolio = np.dot(self.rendimientos, pesos)
        VaR = np.percentile(rendimientos_portafolio, alpha * 100)
        CVaR = rendimientos_portafolio[rendimientos_portafolio <= VaR].mean()
        return -CVaR  # Negativo porque queremos minimizar el CVaR

    def optimizar(self):
        # Definir la función objetivo basada en la estrategia elegida
        if self.optimization_strategy == 'Minima Varianza':
            self.funcion_objetivo = self.minima_varianza
        elif self.optimization_strategy == 'Omega Ratio':
            # Aquí se asume que Omega Ratio se maximiza, por lo que minimizamos su negativo
            self.funcion_objetivo = lambda pesos: -self.omega_ratio(pesos, self.rf)
        elif self.optimization_strategy == 'Semivarianza':
            self.funcion_objetivo = self.semivarianza
        elif self.optimization_strategy == 'Martingala':
            self.funcion_objetivo = self.martingala
        elif self.optimization_strategy == 'Roy Safety First Ratio':
            self.funcion_objetivo = self.roy_safety_first_ratio
        elif self.optimization_strategy == 'Sortino Ratio':
            self.funcion_objetivo = self.sortino_ratio
        elif self.optimization_strategy == 'CVAR':
            self.funcion_objetivo = self.cvar
        else:
            raise ValueError("Estrategia de optimización no válida.")

        # Ejecutar el modelo de optimización seleccionado
        if self.optimization_model == 'SLSQP':
            self.optimizar_slsqp()
        elif self.optimization_model == 'Monte Carlo':
            self.optimizar_montecarlo()
        elif self.optimization_model == 'COBYLA':
            self.optimizar_cobyla()
        else:
            raise ValueError("Modelo de optimización no válido.")



## Validacion diferentes metodos

## Validacion Minima Varianza

In [20]:
from pypfopt import EfficientFrontier, risk_models, expected_returns

# Obtén los datos históricos de precios
datos = estrategia_optimizada.datos

# Calcula el rendimiento esperado y la matriz de covarianza
mu = expected_returns.mean_historical_return(datos)
S = risk_models.sample_cov(datos)

# Optimiza para la mínima varianza usando PyPortfolioOpt
ef = EfficientFrontier(mu, S, weight_bounds=(0.10,1))
pesos_pypfopt = ef.min_volatility()
ef.portfolio_performance(verbose=True)

# Compara los pesos óptimos
print("\nPesos óptimos (Libreria: PyPortfolioOpt):")
print(pesos_pypfopt)

tickers = ["ABBV", "MET", "OXY", "PERI"]
estrategias = ['Minima Varianza']
modelos_optimizacion = ['SLSQP', 'Monte Carlo', 'COBYLA']

# Instanciación de la estrategia optimizada
estrategia_optimizada = EstrategiaOptimizada(tickers=tickers, start_date='2020-01-02', end_date='2024-01-23', rf=0.02)

for estrategia in estrategias:
    estrategia_optimizada.set_optimization_strategy(estrategia)
    for modelo in modelos_optimizacion:
        estrategia_optimizada.set_optimization_model(modelo)
        estrategia_optimizada.optimizar()
        if estrategia_optimizada.pesos_optimos is not None:
            print(f"\nPesos óptimos en nuestra clase({estrategia} - {modelo}):", [format(peso, '.4f') for peso in estrategia_optimizada.pesos_optimos])
        else:
            print(f"\nLa optimización para {estrategia} con {modelo} no convergió o no está implementada.")

Expected annual return: 21.9%
Annual volatility: 25.3%
Sharpe Ratio: 0.79

Pesos óptimos (Libreria: PyPortfolioOpt):
OrderedDict([('ABBV', 0.7), ('MET', 0.1), ('OXY', 0.1), ('PERI', 0.1)])
[*********************100%***********************]  4 of 4 completed

Pesos óptimos en nuestra clase(Minima Varianza - SLSQP): ['0.7000', '0.1000', '0.1000', '0.1000']

Pesos óptimos en nuestra clase(Minima Varianza - Monte Carlo): ['0.7563', '0.0899', '0.0771', '0.0767']

Pesos óptimos en nuestra clase(Minima Varianza - COBYLA): ['0.7976', '0.1492', '0.0000', '0.0532']


## Validacion OMEGA

In [21]:
rf = 0.02
# Función para calcular el Omega Ratio
def omega_ratio(weights, returns, rf):
    portfolio_returns = np.dot(returns, weights)
    excess_returns = portfolio_returns - rf
    upside_potential = np.sum(excess_returns[excess_returns > 0])
    downside_risk = np.sqrt(np.sum(np.square(excess_returns[excess_returns < 0])))
    return upside_potential / downside_risk if downside_risk != 0 else np.inf

# Función objetivo para la optimización (negativo del Omega Ratio)
def neg_omega_ratio(weights, returns, rf):
    return omega_ratio(weights, returns, rf)

# Carga de datos y cálculo de rendimientos
tickers = ["ABBV", "MET", "OXY", "PERI"]
data = yf.download(tickers, start='2020-01-02', end='2024-01-23')['Adj Close']
returns = data.pct_change().dropna().values
rf = 0.02 / 252  # Tasa libre de riesgo diaria

# Restricciones y límites para la optimización
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = [(0.10, 1) for _ in tickers]

# Optimización para maximizar el Omega Ratio
initial_weights = np.ones(len(tickers)) / len(tickers)
result = minimize(neg_omega_ratio, initial_weights, args=(returns, rf), method='SLSQP', bounds=bounds, constraints=constraints)

# Pesos óptimos
optimal_weights = result.x
print("Pesos óptimos para maximizar el Omega Ratio validacion:")
print(optimal_weights)

tickers = ["ABBV", "MET", "OXY", "PERI"]
estrategias = ['Omega Ratio']
modelos_optimizacion = ['SLSQP', 'Monte Carlo', 'COBYLA']

# Instanciación de la estrategia optimizada
estrategia_optimizada = EstrategiaOptimizada(tickers=tickers, start_date='2020-01-02', end_date='2024-01-23', rf=0.02)

for estrategia in estrategias:
    estrategia_optimizada.set_optimization_strategy(estrategia)
    for modelo in modelos_optimizacion:
        estrategia_optimizada.set_optimization_model(modelo)
        estrategia_optimizada.optimizar()
        if estrategia_optimizada.pesos_optimos is not None:
            print(f"\nPesos óptimos en nuestra clase({estrategia} - {modelo}):", [format(peso, '.4f') for peso in estrategia_optimizada.pesos_optimos])
        else:
            print(f"\nLa optimización para {estrategia} con {modelo} no convergió o no está implementada.")

[*********************100%***********************]  4 of 4 completed
Pesos óptimos para maximizar el Omega Ratio validacion:
[0.1 0.1 0.7 0.1]
[*********************100%***********************]  4 of 4 completed

Pesos óptimos en nuestra clase(Omega Ratio - SLSQP): ['0.1000', '0.1000', '0.7000', '0.1000']

Pesos óptimos en nuestra clase(Omega Ratio - Monte Carlo): ['0.0793', '0.0792', '0.0797', '0.7617']

Pesos óptimos en nuestra clase(Omega Ratio - COBYLA): ['-0.0000', '-0.0000', '1.0000', '-0.0000']


## Validacion SEMIVARIANZA NO CUADRA DE MOMENTO

In [22]:
# Función para calcular la semivarianza
def semivariance(returns):
    negative_returns = np.minimum(returns, 0)
    return np.mean(np.square(negative_returns))

# Función objetivo para minimizar la semivarianza del portafolio
def semivariance_portfolio(weights, returns):
    portfolio_returns = np.dot(returns, weights)
    return semivariance(portfolio_returns)

# Restricciones y límites para la optimización
def weight_sum_constraint(weights):
    return np.sum(weights) - 1

# Carga de datos y cálculo de rendimientos
tickers = ["ABBV", "MET", "OXY", "PERI"]
data = yf.download(tickers, start='2020-01-02', end='2024-01-23')['Adj Close']
returns = data.pct_change().dropna().values
rf = 0.02 / 252  # Tasa libre de riesgo diaria

# Parámetros de optimización
initial_weights = np.ones(len(tickers)) / len(tickers)
bounds = [(0.1, 1) for _ in tickers]
constraints = ({'type': 'eq', 'fun': weight_sum_constraint})

# Optimización para minimizar la semivarianza
result = minimize(semivariance_portfolio, initial_weights, args=(returns,), method='SLSQP', bounds=bounds, constraints=constraints)

# Pesos óptimos
optimal_weights = result.x
print("Pesos óptimos para minimizar la semivarianza:")
print(optimal_weights)

# Semivarianza
tickers = ["ABBV", "MET", "OXY", "PERI"]
estrategias = ['Semivarianza']
modelos_optimizacion = ['SLSQP', 'Monte Carlo', 'COBYLA']

# Instanciación de la estrategia optimizada
estrategia_optimizada = EstrategiaOptimizada(tickers=tickers, start_date='2020-01-02', end_date='2024-01-23', rf=0.02)

for estrategia in estrategias:
    estrategia_optimizada.set_optimization_strategy(estrategia)
    for modelo in modelos_optimizacion:
        estrategia_optimizada.set_optimization_model(modelo)
        estrategia_optimizada.optimizar()
        if estrategia_optimizada.pesos_optimos is not None:
            print(f"\nPesos óptimos en nuestra clase ({estrategia} - {modelo}):", [format(peso, '.4f') for peso in estrategia_optimizada.pesos_optimos])
        else:
            print(f"\nLa optimización para {estrategia} con {modelo} no convergió o no está implementada.")


[*********************100%***********************]  4 of 4 completed
Pesos óptimos para minimizar la semivarianza:
[0.25 0.25 0.25 0.25]
[*********************100%***********************]  4 of 4 completed

Pesos óptimos en nuestra clase (Semivarianza - SLSQP): ['0.7000', '0.1000', '0.1000', '0.1000']

Pesos óptimos en nuestra clase (Semivarianza - Monte Carlo): ['0.7667', '0.0781', '0.0775', '0.0777']

Pesos óptimos en nuestra clase (Semivarianza - COBYLA): ['0.8186', '0.1386', '-0.0000', '0.0429']


In [23]:
# Ejemplo de uso simplificado
tickers = ["ABBV", "MET", "OXY", "PERI"]
estrategias = ['Martingala']
modelos_optimizacion = ['SLSQP', 'Monte Carlo', 'COBYLA']

# Instanciación de la estrategia optimizada
estrategia_optimizada = EstrategiaOptimizada(tickers=tickers, start_date='2020-01-02', end_date='2024-01-23', rf=0.02)

for estrategia in estrategias:
    estrategia_optimizada.set_optimization_strategy(estrategia)
    for modelo in modelos_optimizacion:
        estrategia_optimizada.set_optimization_model(modelo)
        estrategia_optimizada.optimizar()
        if estrategia_optimizada.pesos_optimos is not None:
            print(f"\nPesos óptimos en nuestra clase ({estrategia} - {modelo}):", [format(peso, '.4f') for peso in estrategia_optimizada.pesos_optimos])
        else:
            print(f"\nLa optimización para {estrategia} con {modelo} no convergió o no está implementada.")


[*********************100%***********************]  4 of 4 completed

Pesos óptimos en nuestra clase (Martingala - SLSQP): ['0.2057', '0.5943', '0.1000', '0.1000']

Pesos óptimos en nuestra clase (Martingala - Monte Carlo): ['0.3265', '0.5596', '0.0572', '0.0568']

Pesos óptimos en nuestra clase (Martingala - COBYLA): ['0.2390', '0.7116', '0.0000', '0.0493']


In [24]:
# Ejemplo de uso simplificado
tickers = ["ABBV", "MET", "OXY", "PERI"]
estrategias = ['Roy Safety First Ratio']
modelos_optimizacion = ['SLSQP', 'Monte Carlo', 'COBYLA']

# Instanciación de la estrategia optimizada
estrategia_optimizada = EstrategiaOptimizada(tickers=tickers, start_date='2020-01-02', end_date='2024-01-23', rf=0.02)

for estrategia in estrategias:
    estrategia_optimizada.set_optimization_strategy(estrategia)
    for modelo in modelos_optimizacion:
        estrategia_optimizada.set_optimization_model(modelo)
        estrategia_optimizada.optimizar()
        if estrategia_optimizada.pesos_optimos is not None:
            print(f"\nPesos óptimos en nuestra clase ({estrategia} - {modelo}):", [format(peso, '.4f') for peso in estrategia_optimizada.pesos_optimos])
        else:
            print(f"\nLa optimización para {estrategia} con {modelo} no convergió o no está implementada.")

[*********************100%***********************]  4 of 4 completed

Pesos óptimos en nuestra clase (Roy Safety First Ratio - SLSQP): ['0.5345', '0.1000', '0.1000', '0.2655']


c:\Users\PC\anaconda3\lib\site-packages\scipy\optimize\_optimize.py:284: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "



Pesos óptimos en nuestra clase (Roy Safety First Ratio - Monte Carlo): ['0.6030', '0.0611', '0.0610', '0.2749']

Pesos óptimos en nuestra clase (Roy Safety First Ratio - COBYLA): ['0.6893', '0.0000', '0.0295', '0.2811']


## Validar Sortino Ratio

In [25]:
# Ejemplo de uso simplificado
tickers = ["ABBV", "MET", "OXY", "PERI"]
estrategias = ['Sortino Ratio']
modelos_optimizacion = ['SLSQP', 'Monte Carlo', 'COBYLA']

# Instanciación de la estrategia optimizada
estrategia_optimizada = EstrategiaOptimizada(tickers=tickers, start_date='2020-01-02', end_date='2024-01-23', rf=0.02)

def sortino_ratio_optimization(rendimientos, tasa_libre_riesgo_anual):
    """
    Optimiza los pesos del portafolio para maximizar el Ratio de Sortino.

    Parámetros:
    - rendimientos (pd.DataFrame): DataFrame de rendimientos diarios.
    - tasa_libre_riesgo_anual (float): Tasa libre de riesgo anualizada.

    Retorna:
    - np.array: Pesos óptimos del portafolio.
    """
    tasa_libre_riesgo_diaria = tasa_libre_riesgo_anual / 252

    def neg_sortino_ratio(weights):
        portfolio_returns = np.dot(rendimientos, weights)
        downside_returns = np.minimum(portfolio_returns - tasa_libre_riesgo_diaria, 0)
        downside_deviation = np.sqrt(np.mean(downside_returns ** 2))
        sortino_ratio = (np.mean(portfolio_returns) - tasa_libre_riesgo_diaria) / downside_deviation
        return -sortino_ratio  # Negativo porque queremos maximizar

    n_assets = rendimientos.shape[1]
    bounds = [(0.10, 1) for _ in range(n_assets)]
    constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}

    result = minimize(neg_sortino_ratio, np.ones(n_assets) / n_assets, bounds=bounds, constraints=constraints)

    if result.success:
        return result.x
    else:
        raise ValueError("La optimización no fue exitosa.")

# Ejemplo de uso
pesos_optimos = sortino_ratio_optimization(rendimientos, 0.02)
print("Pesos óptimos para maximizar el Ratio de Sortino:")
print(pesos_optimos)

for estrategia in estrategias:
    estrategia_optimizada.set_optimization_strategy(estrategia)
    for modelo in modelos_optimizacion:
        estrategia_optimizada.set_optimization_model(modelo)
        estrategia_optimizada.optimizar()
        if estrategia_optimizada.pesos_optimos is not None:
            print(f"\nPesos óptimos en nuestra clase ({estrategia} - {modelo}):", [format(peso, '.4f') for peso in estrategia_optimizada.pesos_optimos])
        else:
            print(f"\nLa optimización para {estrategia} con {modelo} no convergió o no está implementada.")

[*********************100%***********************]  4 of 4 completed
Pesos óptimos para maximizar el Ratio de Sortino:
[0.44280603 0.1        0.1        0.35719397]

Pesos óptimos en nuestra clase (Sortino Ratio - SLSQP): ['0.4292', '0.1000', '0.1000', '0.3708']

Pesos óptimos en nuestra clase (Sortino Ratio - Monte Carlo): ['0.4961', '0.0502', '0.0503', '0.4034']

Pesos óptimos en nuestra clase (Sortino Ratio - COBYLA): ['0.6062', '0.0000', '0.0094', '0.3845']


## Validacion CVAR

In [26]:
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt.efficient_frontier import EfficientCVaR
import yfinance as yf

# Carga de datos
tickers = ["ABBV", "MET", "OXY", "PERI"]
datos = yf.download(tickers, start="2020-01-01", end="2023-01-01")["Adj Close"]
rendimientos = datos.pct_change().dropna()

# Calcula el rendimiento esperado y la matriz de covarianza
mu = expected_returns.mean_historical_return(datos)
S = risk_models.sample_cov(datos)

# Optimización basada en CVaR
ef_cvar = EfficientCVaR(mu, rendimientos, weight_bounds=(0.1, 1))
pesos_cvar = ef_cvar.min_cvar()
print("Pesos óptimos (CVaR):")
print(pesos_cvar)


# Ejemplo de uso simplificado
tickers = ["ABBV", "MET", "OXY", "PERI"]
estrategias = ['CVAR']
modelos_optimizacion = ['SLSQP', 'Monte Carlo', 'COBYLA']

# Instanciación de la estrategia optimizada
estrategia_optimizada = EstrategiaOptimizada(tickers=tickers, start_date='2020-01-02', end_date='2024-01-23', rf=0.02)

for estrategia in estrategias:
    estrategia_optimizada.set_optimization_strategy(estrategia)
    for modelo in modelos_optimizacion:
        estrategia_optimizada.set_optimization_model(modelo)
        estrategia_optimizada.optimizar()
        if estrategia_optimizada.pesos_optimos is not None:
            print(f"\nPesos óptimos en nuestra clase ({estrategia} - {modelo}):", [format(peso, '.4f') for peso in estrategia_optimizada.pesos_optimos])
        else:
            print(f"\nLa optimización para {estrategia} con {modelo} no convergió o no está implementada.")

[*********************100%***********************]  4 of 4 completed
Pesos óptimos (CVaR):
OrderedDict([('ABBV', 0.6999999999999857), ('MET', 0.1000000000000077), ('OXY', 0.0999999999999994), ('PERI', 0.1000000000000074)])


c:\Users\PC\anaconda3\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


[*********************100%***********************]  4 of 4 completed

Pesos óptimos en nuestra clase (CVAR - SLSQP): ['0.7000', '0.1000', '0.1000', '0.1000']

Pesos óptimos en nuestra clase (CVAR - Monte Carlo): ['0.7307', '0.1206', '0.0741', '0.0745']

Pesos óptimos en nuestra clase (CVAR - COBYLA): ['0.7860', '0.1800', '0.0000', '0.0340']
